In [27]:
%%time
import numpy as np
import pandas as pd
import re
import os
import sys
import nltk
import datetime

from collections import Counter
from nltk.corpus import stopwords

Wall time: 0 ns


In [28]:
def dts():
    return datetime.datetime.now().strftime("%m%d%Y_%H%M%S")

In [29]:
dts()

'12252019_110600'

In [2]:
stop = set(stopwords.words('english'))

In [3]:
stop.add('object')

In [4]:
#Read text data
df = pd.read_table('../data/raw/dictionary.txt')

In [5]:
df.head()

,!|0
0,! '|22935
1,! ''|18235
2,! Alas|179257
3,! Brilliant|22936
4,! Brilliant !|40532


In [6]:
df[['Text','text_id']] = df['!|0'].str.split('|',expand=True)

In [7]:
df.head()

,!|0,Text,text_id
0,! '|22935,! ',22935
1,! ''|18235,! '',18235
2,! Alas|179257,! Alas,179257
3,! Brilliant|22936,! Brilliant,22936
4,! Brilliant !|40532,! Brilliant !,40532


In [8]:
df.describe()

,!|0,Text,text_id
count,239231,239231,239231
unique,239231,239231,239231
top,that team up for a|213861,will enjoy a fast-paced comedy with quirks tha...,221811
freq,1,1,1


In [9]:
#Read Sentiment labels 0-Most Negative, 1-Most positive
df_sent = pd.read_table('../data/raw/sentiment_labels.txt')

In [10]:
df_sent.head()

,phrase ids|sentiment values
0,0|0.5
1,1|0.5
2,2|0.44444
3,3|0.5
4,4|0.42708


In [11]:
df_sent[['text_id','sentiment']] = df_sent['phrase ids|sentiment values'].str.split('|',expand=True)

In [12]:
df_sent.head()

,phrase ids|sentiment values,text_id,sentiment
0,0|0.5,0,0.5
1,1|0.5,1,0.5
2,2|0.44444,2,0.44444
3,3|0.5,3,0.5
4,4|0.42708,4,0.42708


In [13]:
df_final = df.merge(df_sent,how='inner',on='text_id')

In [14]:
df_final.head(15)

,!|0,Text,text_id,phrase ids|sentiment values,sentiment
0,! '|22935,! ',22935,22935|0.52778,0.52778
1,! ''|18235,! '',18235,18235|0.5,0.5
2,! Alas|179257,! Alas,179257,179257|0.44444,0.44444
3,! Brilliant|22936,! Brilliant,22936,22936|0.86111,0.86111
4,! Brilliant !|40532,! Brilliant !,40532,40532|0.93056,0.93056
5,! Brilliant ! '|22937,! Brilliant ! ',22937,22937|1,1
6,! C'mon|60624,! C'mon,60624,60624|0.47222,0.47222
7,! Gollum 's ` performance ' is incredible|13402,! Gollum 's ` performance ' is incredible,13402,13402|0.76389,0.76389
8,"! Oh , look at that clever angle ! Wow , a jum...","! Oh , look at that clever angle ! Wow , a jum...",179258,179258|0.27778,0.27778
9,! Romething|140882,! Romething,140882,140882|0.5,0.5


In [15]:
df_final.shape

(239231, 5)

In [16]:
df_new = df_final[['text_id','Text','sentiment']].copy()

In [17]:
def clean(text):
    """
    :param text: tweet text
    :return proc_text: processed text
    remove stopwords, lower case all words, remove punctuations, stemming words
    """
    text = re.sub(r'[^a-zA-Z]',' ',text)
    text = re.sub(r'Text | dtype | object | Name',' ',text)
    text = re.sub(r'\s{1,}',' ',text)
    words = text.split()
    all_words = [str(word).lower() for word in words if word not in stop]
    all_words = ' '.join(all_words)
    return all_words

In [18]:
%%time
df_new['processed_text'] = df_new[['Text']].apply(lambda x: clean(str(x)), axis=1)

Wall time: 2min 57s


In [20]:
vocab = token(df_new['processed_text'])

NameError: name 'token' is not defined

In [115]:
len(vocab)

958361

In [116]:
len(set(vocab))

263229

In [19]:
df_new.head()

,text_id,Text,sentiment,processed_text
0,22935,! ',0.52778,
1,18235,! '',0.5,
2,179257,! Alas,0.44444,alas
3,22936,! Brilliant,0.86111,brilliant
4,40532,! Brilliant !,0.93056,brilliant


In [20]:
x = 3
np.where(x<5,0,1)

array(0)

In [21]:
def find_polarity(value):
    if value < 0.5:
        return 0
    else:
        return 1    

In [22]:
df_new.dtypes

text_id           object
Text              object
sentiment         object
processed_text    object
dtype: object

In [23]:
df_new['sentiment'] = df_new[['sentiment']].apply(pd.to_numeric)

In [24]:
df_new.dtypes

text_id            object
Text               object
sentiment         float64
processed_text     object
dtype: object

In [25]:
df_new['Polarity'] = df_new.sentiment > 0.5

In [26]:
df_new.head()

,text_id,Text,sentiment,processed_text,Polarity
0,22935,! ',0.52778,,True
1,18235,! '',0.50000,,False
2,179257,! Alas,0.44444,alas,False
3,22936,! Brilliant,0.86111,brilliant,True
4,40532,! Brilliant !,0.93056,brilliant,True


In [30]:
%%time
df_new.to_excel('../data/processed/twitter_corpus_{}.xlsx'.format(dts()))